In [99]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [49]:
stimuli = pd.read_csv('../../stimuli_specificity_2023-05-25.csv')[['set_nr','specificity','word1','word2']]

Extract specificity and items 

In [50]:
high = stimuli.query('specificity == "high"')[['set_nr','specificity','word2']].rename(columns={'word2':'option'})
mid = stimuli.query('specificity == "mid"')[['set_nr','specificity','word1','word2']]
mid['phrase'] = mid['word1'] + ' ' + mid['word2']
mid = mid[['set_nr','specificity','phrase']].rename(columns={'phrase':'option'})
low = stimuli.query('specificity == "low"')[['set_nr','specificity','word2']].rename(columns={'word2':'option'})
stimuli = pd.concat([high, mid, low])
stimuli

,set_nr,specificity,option,condition
0,1,high,kangaroo,
2,1,low,animal,
1,1,mid,hopping animal,
3,2,high,snail,
5,2,low,animal,
...,...,...,...,...
297,100,high,pasta,
298,100,mid,Italian food,
301,101,mid,Japanese food,
300,101,high,sushi,


Build pairs of options

In [202]:
options_new = []
specificity_new = []
set_nr_new = []
stimuli_shuffled = shuffle(stimuli, random_state=42) # shuffle everything 
for i_set in stimuli_shuffled.set_nr.unique():
    stimuli_subset = stimuli_shuffled.query('set_nr == @i_set').reset_index(drop=True)
    for combination in [[0,1],[1,2],[0,2]]:
        options_new.append(stimuli_subset.iloc[combination].option.tolist())
        specificity_new.append(stimuli_subset.iloc[combination].specificity.tolist())
        set_nr_new.append(i_set)
options_new = np.array(options_new).transpose()
stimuli_new = pd.DataFrame({'set_nr': set_nr_new,
                            'specificity': specificity_new,
                            'option1': options_new[0],
                            'option2': options_new[1]})
stimuli_new = stimuli_new.sort_values(by='set_nr').reset_index(drop=True)
stimuli_new['item_nr'] = stimuli_new.index + 1
stimuli_new = stimuli_new[stimuli_new.columns.tolist()[-1:] + stimuli_new.columns.tolist()[:-1]]
stimuli_new

,item_nr,set_nr,specificity,option1,option2
0,1,1,"[low, mid]",animal,hopping animal
1,2,1,"[mid, high]",hopping animal,kangaroo
2,3,1,"[low, high]",animal,kangaroo
3,4,2,"[high, mid]",snail,slow animal
4,5,2,"[high, low]",snail,animal
...,...,...,...,...,...
298,299,100,"[low, mid]",food,Italian food
299,300,100,"[high, mid]",pasta,Italian food
300,301,101,"[mid, low]",Japanese food,food
301,302,101,"[high, low]",sushi,food


Make gorilla spreadsheet: counterbalancing of choosing the less or more specific item in a pair

In [205]:
specificity_mapping = {'low':1, 'mid':2, 'high':3}
stimuli_new_with_answers = stimuli_new
stimuli_new_with_answers['answer'] = ''
for answer in ['less','more']:
    answers = []
    for row in stimuli_new.iterrows():
        row = row[1]
        if answer == 'less':
            if specificity_mapping[row.specificity[0]] < specificity_mapping[row.specificity[1]]:
                answers.append('left')
            else:
                answers.append('right')
        elif answer == 'more':
            if specificity_mapping[row.specificity[0]] < specificity_mapping[row.specificity[1]]:
                answers.append('right')
            else:
                answers.append('left')
    stimuli_new_with_answers['answer'] = answers


     item_nr  set_nr  specificity         option1         option2 answer
0          1       1   [low, mid]          animal  hopping animal   left
1          2       1  [mid, high]  hopping animal        kangaroo   left
2          3       1  [low, high]          animal        kangaroo   left
3          4       2  [high, mid]           snail     slow animal  right
4          5       2  [high, low]           snail          animal  right
..       ...     ...          ...             ...             ...    ...
298      299     100   [low, mid]            food    Italian food   left
299      300     100  [high, mid]           pasta    Italian food  right
300      301     101   [mid, low]   Japanese food            food  right
301      302     101  [high, low]           sushi            food  right
302      303     101  [high, mid]           sushi   Japanese food  right

[303 rows x 6 columns]
     item_nr  set_nr  specificity         option1         option2 answer
0          1       1   [low